# Mid Level IR


Philip Zucker recently wrote a blog post about using egglog to do simplifications by converting from the Z3 API, ["Conditional Simplification of Z3py Expressions with Egglog"](https://www.philipzucker.com/egglog_z3_simp/).

In it, he uses the egglog bindings, but only the low level bindings, as a way to execute egglog programs as strings. So he constructs expressions and rewrites using the Z3 API, manually converts them to egglog program strings, runs them in egglog, and then gets them back.

He prefers to use the Z3 API over the high level egglog API, because it is a simpler functional interface, creating sorts and functions through functions, instead of by using Python classes:

> Saul has been making the egglog python bindings https://egglog-python.readthedocs.io/latest/ taking a very meta highly integrated approach. I kind of just want it to look like z3 though. It’s very interesting and I’m haunted by the idea that I am a stodgy old man and they’re right. I think it’s biggest demerit is that it is very novel. I’ve never seen an interface like it. From a research perspective this is a plus. It is very cool that they are getting the python typechecker and the embedded dsl to play ball. I dunno https://egraphs.zulipchat.com/#narrow/stream/375765-egg.2Fegglog/topic/egglog.20python.20midlevel.20api/near/421919681
>
> It turns out, it is simple enough to have my cake too. The pyegglog supports the raw bindings and I’ve been spending a decent amount of time serializing Z3 ASTs to other smt or tptp solvers. Translating to egglog programs is easy.

In order to continue this conversation, I added two functions `sort` and `fn`, to the egglog bindings, to see how those could be used to do something similar. The overall hope is that we could re-use some of the existing mechanisms in the high level API, even with a slightly different external interface. Here is an example of using them to re-create the example Phil wrote up:


In [4]:
from egglog import *

Math = sort("Math")
add = fn("add", Math, Math, Math)
zero = constant("zero", Math)
one = constant("one", Math)

sig = [Math, add, zero, one]

x, y = vars_("x y", Math)

rules = ruleset(
    rewrite(add(x, zero)).to(x),
    rewrite(add(x, y)).to(add(y, x)),
)

res = simplify(add(zero, one), rules.saturate())
res

one

## Metaprogramming in egglog

What about if you want to do more generic metaprogramming in egglog? So far, I have tried to keep the internal details of how egglog objects are stored hidden from external users, so that they can be changed without breaking the API. However, if you do want to try doing some metaprogramming, the API has been written in such a way that each egglog object will be first converted into data only "declerations" before then being converted into the egglog string.

These "declerations" are what are stored at runtime. So it's actually quite easy to manually create whatever objects (expressions, sorts, functions, rewrite, etc) that you want just using the normal data classes, and then convert them to egglog strings. Or to go the other way, and match on them to convert them to another syntax. Again, this intentionally has not been part of the public API yet, because things are still unstable, but was designed in such a way to make metaprogramming easier.

For example, we can start by looking at the expression object. At runtime, it's simply a dataclass with two fields, the `Declerations`, like the total state of the e-graph we need to know, and the `expr`, a pointer into that state of the expressions:


In [14]:
from rich.pretty import pprint

x = add(zero, one)

pprint(x.__egg_typed_expr__)
pprint(x.__egg_decls__)

TypedExprDecl(
│   tp=JustTypeRef(name='Math', args=()),
│   expr=CallDecl(
│   │   callable=FunctionRef(name='add'),
│   │   args=(
│   │   │   TypedExprDecl(
│   │   │   │   tp=JustTypeRef(name='Math', args=()),
│   │   │   │   expr=CallDecl(callable=ConstantRef(name='zero'), args=(), bound_tp_params=None)
│   │   │   ),
│   │   │   TypedExprDecl(
│   │   │   │   tp=JustTypeRef(name='Math', args=()),
│   │   │   │   expr=CallDecl(callable=ConstantRef(name='one'), args=(), bound_tp_params=None)
│   │   │   )
│   │   ),
│   │   bound_tp_params=None
│   )
)

Declarations(
│   _functions={
│   │   'add': FunctionDecl(
│   │   │   signature=FunctionSignature(
│   │   │   │   arg_types=(TypeRefWithVars(name='Math', args=()), TypeRefWithVars(name='Math', args=())),
│   │   │   │   arg_names=('__0', '__1'),
│   │   │   │   arg_defaults=(None, None),
│   │   │   │   return_type=TypeRefWithVars(name='Math', args=()),
│   │   │   │   var_arg_type=None
│   │   │   ),
│   │   │   builtin=False,
│   │   │   egg_name=None,
│   │   │   cost=None,
│   │   │   default=None,
│   │   │   on_merge=(),
│   │   │   merge=None,
│   │   │   unextractable=False
│   │   )
│   },
│   _constants={
│   │   'zero': ConstantDecl(type_ref=JustTypeRef(name='Math', args=()), egg_name=None),
│   │   'one': ConstantDecl(type_ref=JustTypeRef(name='Math', args=()), egg_name=None)
│   },
│   _classes={
│   │   'Math': ClassDecl(
│   │   │   egg_name=None,
│   │   │   type_vars=(),
│   │   │   builtin=False,
│   │   │   init=None,
│   │   │   class_methods={},
│   │   │   class_variables={},
│   │   │   methods={},
│   │   │   properties={},
│   │   │   preserved_methods={}
│   │   )
│   },
│   _rulesets={'': RulesetDecl(rules=[])}
)

The same is true of the rulesets:


In [17]:
pprint(rules.__egg_name__)
pprint(rules.__egg_decls__)

'ruleset_4743265072'

Declarations(
│   _functions={
│   │   'add': FunctionDecl(
│   │   │   signature=FunctionSignature(
│   │   │   │   arg_types=(TypeRefWithVars(name='Math', args=()), TypeRefWithVars(name='Math', args=())),
│   │   │   │   arg_names=('__0', '__1'),
│   │   │   │   arg_defaults=(None, None),
│   │   │   │   return_type=TypeRefWithVars(name='Math', args=()),
│   │   │   │   var_arg_type=None
│   │   │   ),
│   │   │   builtin=False,
│   │   │   egg_name=None,
│   │   │   cost=None,
│   │   │   default=None,
│   │   │   on_merge=(),
│   │   │   merge=None,
│   │   │   unextractable=False
│   │   )
│   },
│   _constants={'zero': ConstantDecl(type_ref=JustTypeRef(name='Math', args=()), egg_name=None)},
│   _classes={
│   │   'Math': ClassDecl(
│   │   │   egg_name=None,
│   │   │   type_vars=(),
│   │   │   builtin=False,
│   │   │   init=None,
│   │   │   class_methods={},
│   │   │   class_variables={},
│   │   │   methods={},
│   │   │   properties={},
│   │   │   preserved_methods={}
│   │   )
│   },
│   _rulesets={
│   │   '': RulesetDecl(rules=[]),
│   │   'ruleset_4743265072': RulesetDecl(
│   │   │   rules=[
│   │   │   │   RewriteDecl(
│   │   │   │   │   tp=JustTypeRef(name='Math', args=()),
│   │   │   │   │   lhs=CallDecl(
│   │   │   │   │   │   callable=FunctionRef(name='add'),
│   │   │   │   │   │   args=(
│   │   │   │   │   │   │   TypedExprDecl(tp=JustTypeRef(name='Math', args=()), expr=VarDecl(name='x')),
│   │   │   │   │   │   │   TypedExprDecl(
│   │   │   │   │   │   │   │   tp=JustTypeRef(name='Math', args=()),
│   │   │   │   │   │   │   │   expr=CallDecl(callable=ConstantRef(name='zero'), args=(), bound_tp_params=None)
│   │   │   │   │   │   │   )
│   │   │   │   │   │   ),
│   │   │   │   │   │   bound_tp_params=None
│   │   │   │   │   ),
│   │   │   │   │   rhs=VarDecl(name='x'),
│   │   │   │   │   conditions=(),
│   │   │   │   │   subsume=False
│   │   │   │   ),
│   │   │   │   RewriteDecl(
│   │   │   │   │   tp=JustTypeRef(name='Math', args=()),
│   │   │   │   │   lhs=CallDecl(
│   │   │   │   │   │   callable=FunctionRef(name='add'),
│   │   │   │   │   │   args=(
│   │   │   │   │   │   │   TypedExprDecl(tp=JustTypeRef(name='Math', args=()), expr=VarDecl(name='x')),
│   │   │   │   │   │   │   TypedExprDecl(tp=JustTypeRef(name='Math', args=()), expr=VarDecl(name='y'))
│   │   │   │   │   │   ),
│   │   │   │   │   │   bound_tp_params=None
│   │   │   │   │   ),
│   │   │   │   │   rhs=CallDecl(
│   │   │   │   │   │   callable=FunctionRef(name='add'),
│   │   │   │   │   │   args=(
│   │   │   │   │   │   │   TypedExprDecl(tp=JustTypeRef(name='Math', args=()), expr=VarDecl(name='y')),
│   │   │   │   │   │   │   TypedExprDecl(tp=JustTypeRef(name='Math', args=()), expr=VarDecl(name='x'))
│   │   │   │   │   │   ),
│   │   │   │   │   │   bound_tp_params=None
│   │   │   │   │   ),
│   │   │   │   │   conditions=(),
│   │   │   │   │   subsume=False
│   │   │   │   )
│   │   │   ]
│   │   )
│   }
)

When we then run a ruleset, or add an expression, the declerations are converted to egglog strings and run. If we want to manually parse the expression, we could match on the different types of declerations:


In [27]:
from egglog.declarations import *


def expr_to_string(expr: TypedExprDecl) -> str:
    """
    Recursively convert a typed expression into a string, to show how to traverse them
    """
    tp = tp_to_string(expr.tp)
    match expr.expr:
        case CallDecl(f, args, _):
            f_str = callable_ref_to_string(f)
            args_str = ", ".join(expr_to_string(a) for a in args)
            expr_str = f"{f_str}({args_str})"
        case _:
            raise NotImplementedError(f"TypedExprDecl {type(expr)} not implemented")

    return f"cast({tp}, {expr_str})"


def tp_to_string(tp: JustTypeRef) -> str:
    name = tp.name
    if tp.args:
        args = ", ".join(tp_to_string(a) for a in tp.args)
        return f"{name}[{args}]"
    return name


def callable_ref_to_string(call: CallableRef) -> str:
    match call:
        case FunctionRef(name):
            return name
        case ConstantRef(name):
            return name
        case _:
            raise NotImplementedError(f"CallableRef {call} not implemented")


expr_to_string(x.__egg_typed_expr__)

'cast(Math, add(cast(Math, zero()), cast(Math, one())))'

This gives a sense of how you can use metaprogramming to construct or deconstruct egglog objects. If there are other protocols like z3, we could add support for using metaprogramming to go back and forth to and from egglog and those as well.
